In [1]:
from attrs_per_mEPSP import *
%load_ext autoreload
%autoreload 2

### Goal

This notebook demonstrates how to regenerate the data underlying Figure 5: *Arbors democratize and cables stratify synaptic efficacy* of the paper **Connectomic features underlying diverse synaptic connection strengths and subcellular computation**. In particular, the code uses the NEURON simulation engine, accessed via Python, to iterate through a list of lateral horn neurons, initializing each into NEURON and simulating attributes of each incoming synapse -- attributes including the miniEPSP amplitude, impedance properties, morphological properties (i.e. distances to the putative spike initiation zone, pSIZ), and more. 

Other code functionalities:
- The function `visualize_inputs` can be used to instantiate a single neuron into the NEURON workspace, which can then be used for simulating unitary EPSPs or mini EPSPs, manipulating synapse locations, perturbing morphology, and more. 
- Please feel free to reach out to the authors at any point with questions about the code!

### `attrs_per_LHNmEPSP` demonstration

For this example, we will simulate data for synapses onto a local lateral horn neuron (local5, hemibrain ID 5813105722) and an output lateral horn neuron (ML9, 542634516). Notably, to simulate data for all neurons in the paper, set `only_these_body_IDs` to be an empty string and set `weight_threshold` to 0 -- this will likely run for many hours (close to a day). 

Further, we can perform morphological manipulations prior to evaluating synapse attributes. In the paper, we compared the base model (unperturbed morphologies) to perturbations where (1) the dendritic and axonal arbors were made isopotential -- to study the location-dependent effects of synaptic efficacy *within* arbors -- and where (2) the inter-arbor cable was made isopotential -- to study the influence of the inter-arbor cable at stratifying synaptic efficacies *between* arbors. These manipulations corresponded to changes in the following parameters:
1.  `'isopot_dendr_arbor': True, 'isopot_ax_arbor': True`
2. `'isopot_prim_ax': True, 'isopot_prim_dendr': True`

In [6]:
help(attrs_per_LHNmEPSP)

Help on function attrs_per_LHNmEPSP in module attrs_per_mEPSP:

attrs_per_LHNmEPSP(nrn_list_path='21-05-07_LHN_SWCPointNo_to_NEURON.csv', weight_threshold=0, transf_freq=0, input_name='all_ePN_inputs', LHLNs_only=False, only_these_body_IDs=[], use_var_time_step=False, fixed_time_step=0.06, compt_manips={'isopot_dendr_arbor': False, 'isopot_ax_arbor': False, 'isopot_prim_ax': False, 'isopot_prim_dendr': False, 'isopot_whole_nrn': False, 'isopot_Ra_val': 0.001, 'highpot_prim_ax': False, 'highpot_Ra_val': 10000, 'IAC_uniform_diam': False, 'IAC_no_pinch': False, 'IAC_pinch_thresh': 0.14, 'IAC_eucl_dist': False, 'IAC_3x_len': False}, params={'R_a': 350, 'c_m': 0.6, 'g_pas': 5.8e-05, 'e_pas': -55, 'syn_strength': 5.5e-05}, param_str='')
    for each neuron in a list (i.e. a list of LHNs), find information about EACH mEPSP of its
            input connections (for all inputs >weight_threshold synapses at all 
            locations on the neuron), such as mEPSP amplitude, impedance measures, s

In [2]:
base_df = attrs_per_LHNmEPSP(nrn_list_path = '21-05-07_LHN_SWCPointNo_to_NEURON.csv',
                             weight_threshold = 50, 
                             transf_freq = 0,
                             input_name = 'all_ePN_inputs', 
                             LHLNs_only = False, 
                             only_these_body_IDs = [5813105722, 542634516],
                             use_var_time_step = False, 
                             fixed_time_step = 0.06, 
                             compt_manips = {'isopot_dendr_arbor': False, 'isopot_ax_arbor': False, 
                                             'isopot_prim_ax': False, 'isopot_prim_dendr': False,
                                             'isopot_whole_nrn': False,
                                             'isopot_Ra_val': 0.001,
                                             'highpot_prim_ax': False, 
                                             'highpot_Ra_val': 10000,
                                             'IAC_uniform_diam': False, 
                                             'IAC_no_pinch': False, 'IAC_pinch_thresh': 0.14, 
                                             'IAC_eucl_dist': False,
                                             'IAC_3x_len': False},
                             params = {'R_a': 350, # ohm-cm
                                      'c_m': 0.6, # uF/cm^2
                                      'g_pas': 5.8e-5, # S/cm^2
                                      'e_pas': -55, # mV one parameter left same in both, we used -55 in our fits
                                      'syn_strength': 5.5e-5},
                             param_str = '')

visualizing target: local5 5813105722, input: all_ePN_inputs
One point section Import3d_Section[44] ending at line 75 has been removed
	and child Import3d_Section[45] reattached
	and child Import3d_Section[46] reattached
One point section Import3d_Section[1] ending at line 6 has been removed
	and child Import3d_Section[2] reattached
	and child Import3d_Section[39] reattached
fetching all ePN synapse locations >50 synapses
adding 182 synapses from all_ePN_inputs to local5; budget = 0.26608187134502925
proportion synapses in dendrite: 0.5494505494505495
measure_locs acquired: {'dendr_start': [195, 0.327443109], 'dendr_first_branch': [196, 1.0], 'ax_start': [996, 1.0], 'ax_first_branch': [1002, 1.0], 'ax_distal': [0, 0.5], 'soma': [0, 0.5]}
probing mEPSP for synapse 1
probing mEPSP for synapse 101
soma summation efficacy: 0.6414963112306664
SIZ summation efficacy: 0.6107621106769365
adding imp. measurements to soma
adding imp. measurements to ax_start
adding imp. measurements to dendr_fir

The simulated attributes, per synapse, include the following: 

In [4]:
base_df.columns

Index(['target_name', 'target_body_id', 'input_name', 'input_body_id',
       'synapse_number', 'syn_object', 'syn_loc', 'ax_v_dendr', 'syn_count',
       'local_diam', 'dist_to_siz', 'dist_to_soma',
       'dist_to_dendr_first_branch', 'mEPSP_siz', 'mEPSP_soma',
       'mEPSP_ax_first_branch', 'mEPSP_dendr_start',
       'mEPSP_dendr_first_branch', 'mEPSP_synloc', 'mEPSP_t10to90_siz',
       'mEPSP_t10to90_soma', 'mEPSP_tPNspiketo100_siz',
       'mEPSP_tPNspiketo100_dendr_first_branch',
       'mEPSP_tPNspiketo100_ax_first_branch', 't_trace', 'v_trace_soma',
       'v_trace_siz', 'v_trace_ax_first_branch', 'v_trace_dendr_start',
       'v_trace_dendr_first_branch', 'v_trace_synloc', 't_trace_uEPSP',
       'v_trace_uEPSP_soma', 'v_trace_uEPSP_siz', 'Zi', 'Zc_to_soma',
       'K_to_soma', 'dist_to_ax_start', 'Zc_to_ax_start', 'K_to_ax_start',
       'Zc_to_dendr_first_branch', 'K_to_dendr_first_branch',
       'target_surf_area_total', 'target_surf_area_dendr_arbor',
       'target_su

In [3]:
base_df

,target_name,target_body_id,input_name,input_body_id,synapse_number,syn_object,syn_loc,ax_v_dendr,syn_count,local_diam,...,Zc_to_soma,K_to_soma,dist_to_ax_start,Zc_to_ax_start,K_to_ax_start,Zc_to_dendr_first_branch,K_to_dendr_first_branch,target_surf_area_total,target_surf_area_dendr_arbor,target_surf_area_ax_arbor
0,local5,5813105722,VA6_adPN,1881751117,0,Exp2Syn[0],Cell[0].axon[811](1),dendritic,182,0.140153,...,391.042041,0.396847,23.506362,667.877406,0.677792,682.430885,0.692562,3583.320545,1360.051447,1468.203977
1,local5,5813105722,VA6_adPN,1881751117,1,Exp2Syn[1],Cell[0].axon[429](0.928571),dendritic,182,0.139911,...,389.487606,0.428100,18.578794,665.222520,0.731170,679.718147,0.747103,3583.320545,1360.051447,1468.203977
2,local5,5813105722,VA6_adPN,1881751117,2,Exp2Syn[2],Cell[0].axon[865](0.125),dendritic,182,0.273430,...,385.927487,0.324508,38.576149,659.142040,0.554241,673.505169,0.566318,3583.320545,1360.051447,1468.203977
3,local5,5813105722,VA6_adPN,1881751117,3,Exp2Syn[3],Cell[0].axon[462](0.625),dendritic,182,0.239596,...,386.231133,0.381977,26.824738,659.660650,0.652394,674.035081,0.666610,3583.320545,1360.051447,1468.203977
4,local5,5813105722,VA6_adPN,1881751117,4,Exp2Syn[4],Cell[0].axon[472](0.821429),dendritic,182,0.345234,...,384.900427,0.295306,31.676352,657.387880,0.504365,671.712786,0.515356,3583.320545,1360.051447,1468.203977
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50,ML9,542634516,DM1_lPN,542634818,50,Exp2Syn[232],Cell[0].axon[359](0.1875),dendritic,55,0.150999,...,518.131704,0.371497,38.033389,823.236563,0.590256,890.018257,0.638138,2595.719921,1219.214762,544.582358
51,ML9,542634516,DM1_lPN,542634818,51,Exp2Syn[233],Cell[0].axon[470](1),dendritic,55,0.128000,...,499.470856,0.230720,65.435887,793.587167,0.366581,857.963675,0.396318,2595.719921,1219.214762,544.582358
52,ML9,542634516,DM1_lPN,542634818,52,Exp2Syn[234],Cell[0].axon[104](0.558824),dendritic,55,0.386566,...,479.794760,0.342043,75.903686,762.324688,0.543457,824.165155,0.587542,2595.719921,1219.214762,544.582358
53,ML9,542634516,DM1_lPN,542634818,53,Exp2Syn[235],Cell[0].axon[125](0.875),dendritic,55,0.185116,...,481.350272,0.258563,75.443404,764.796173,0.410820,826.837128,0.444146,2595.719921,1219.214762,544.582358
